In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('s200862/medical_qa_meds', split='train')

In [ ]:
def parse_sample(sample):
    # Extract the prompt
    prompt_start = sample.find('[INST]') + len('[INST]')
    prompt_end = sample.find('[/INST]')
    prompt = sample[prompt_start:prompt_end].strip()
    
    # Extract the symptoms
    symptoms = sample[prompt_end + len('[/INST]'):].strip()
    
    return prompt, symptoms

# Initialize lists to store parsed data
prompts = []
symptoms_list = []

# Parse each sample

parsed_samples = list(map(parse_sample, train_dataset['text']))
prompts, symptoms_list = zip(*parsed_samples)
   

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'Prompt': prompts,
    'Symptoms': symptoms_list
})



In [ ]:
from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(df)

In [ ]:
from sklearn.model_selection import train_test_split

# Function to split dataset into train, validation, and test sets
def split_dataset(dataset, train_size=0.8, val_size=0.1, test_size=0.1):
    assert train_size + val_size + test_size == 1, "Train, val, and test sizes must sum to 1."
    
    train_test_split = dataset.train_test_split(test_size=(val_size + test_size))
    train_set = train_test_split['train']
    
    val_test_split = train_test_split['test'].train_test_split(test_size=test_size / (val_size + test_size))
    val_set = val_test_split['train']
    test_set = val_test_split['test']
    
    return DatasetDict({
        'train': train_set,
        'validation': val_set,
        'test': test_set
    })

# Split the dataset
dataset_dict = split_dataset(dataset, train_size=0.8, val_size=0.1, test_size=0.1)

# Verify the splits
print(dataset_dict)

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

In [ ]:
dataset_dict.push_to_hub("medical_datset")

Now Choose max length for training

def formatting_func(example):
    text = f"### Question: {example['Prompt']}\n ### Answer: {example['Symptoms']}"
    return text

In [ ]:
from datasets import load_dataset

dataset = load_dataset('Prajapat/medical_datset',split = 'train')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [ ]:
tokenized_train_dataset = dataset.map(generate_and_tokenize_prompt)

In [ ]:
import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    # lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

plot_data_lengths(tokenized_train_dataset)